# Filtering

Part of this project will rely on finding the relationship between different frequency components within the O2 data and the LFP data. This notebook will perform Fourier transforms and low/high pass filtering on the O2 data to get a better understanding of the data and Julia libraries.

In [46]:
using Plots
using FFTW
using Fourier
using DSP
using Statistics

## Importing

In [47]:
# Indexing o2_data[datapoint, time/o2]

o2_path = "Exp 1/R4/"
filename = "o2data R4.txt"
o2_data = []
open("../data/$(o2_path)$(filename)") do file
    for line in eachline(file)
        if isdigit(line[1])
            data_element = split(line, "\t")
            data_element = [parse(Float32, data_element[1]), parse(Float32, data_element[2])]
            push!(o2_data, data_element)
        end
    end
end
o2_data = reduce(vcat,transpose.(o2_data))
o2_data_subset = o2_data[100:10000, :] 
o2_data_subset[:, 2] = (o2_data_subset[:, 2] .- mean(o2_data_subset[:, 2])) ./ var(o2_data_subset[:, 2])
o2_data_subset

In [ ]:
plot(o2_data_subset[:, 1], o2_data_subset[:, 2], legend=false)
title!("O2 Dataset to be used in Forier Analysis")
xlabel!("Time (s)")

## Fourier Transform

This section will attempt to compute and graph the Fourier Space of the data to get a better understanding of the frequency components that it comprises.

In [ ]:
sampling_rate = 1000

F = fft(o2_data_subset[:,2])
freqs = fftfreq(length(o2_data_subset[:,1]), sampling_rate)

In [ ]:
# Plot
re = plot(freqs, real(F), title = "Real", xlim=(-150,150), ylim=(0,maximum(real(F))), legend=false)
im = plot(freqs, imag(F), title = "Imaginary", xlim=(-150,150), ylim=(0,maximum(imag(F))), legend=false)
plot(re,im, layout=layout = grid(1, 2), plot_title="Fourier Space of O2 Data")
xlabel!("Frequency (Hz)")
ylabel!("Amplitude")

## Low Pass Filtering

Objective: extract the low frequency signals from the o2 data (<1Hz).

In [ ]:
pass_freq = 10

low_pass_F = copy(F)
low_pass_F[findall(x->x>pass_freq, freqs)] .= 0
low_pass_F[findall(x->x<-pass_freq, freqs)] .= 0
plot(freqs, abs.(low_pass_F), xlim=(-150,150), ylim=(0,maximum(abs.(low_pass_F))), legend=false)
title!("Low Pass Filtered Fourier Space of O2 Data")
xlabel!("Frequency (Hz)")
ylabel!("Amplitude")

In [ ]:
plot(o2_data_subset[:,2], label="Original")
plot!(real(ifft(low_pass_F)), label="Transformed", linewidth = 1.5)
title!("Low Pass Filtered Fourier Space of O2 Data")
xlabel!("Time (s)")

## High Pass Filtering

Objective: extract the high frequency signals from the o2 data (>1Hz).

In [ ]:
pass_freq = 10

high_pass_F = copy(F)
high_pass_F[findall(x->-pass_freq<x<pass_freq, freqs)] .= 0
plot(freqs, abs.(high_pass_F), xlim=(-150,150), ylim=(0,maximum(abs.(high_pass_F))), legend=false)
title!("High Pass Filtered Fourier Space of O2 Data")
xlabel!("Frequency (Hz)")
ylabel!("Amplitude")

In [ ]:
plot(o2_data_subset[:,2], label="Original")
plot!(real(ifft(high_pass_F)), label="Transformed", linewidth = 1.5)
title!("High Pass Filtered Fourier Space of O2 Data")
xlabel!("Time (s)")